In [29]:
from lxml import etree       
import numpy as np
import pandas as pd
import chardet
import pymorphy2
from sklearn.ensemble import GradientBoostingClassifier
import re
import cPickle as pickle
import os
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from tqdm import tqdm
import json
import io
import scipy
import sys
import gc
import random as r

# Преобразовываем xml в csv

In [104]:
tree = etree.parse('./news_sentiment_romip2012/train/news_eval_train.xml')  
root = tree.getroot() 
corpus = root
data = []

for sentense in corpus:
    new_sentense = []
    new_sentense.append(sentense.items()[0][1])
#     print sentense.getchildren()
    for field in sentense[:-1]:
        if field.text is None:
            new_sentense.append(None)
        else:
            
            new_sentense.append(field.text.encode('utf-8').strip())
    data.append(new_sentense)

Sentences = pd.DataFrame(np.asarray(data), columns = ['id', 'text', 'label'])
Sentences[['id']] = Sentences[['id']].apply(pd.to_numeric) 
Sentences = Sentences[(Sentences.label == "0") | (Sentences.label == "+") | (Sentences.label == "-")]
Sentences['label'] = Sentences['label'].map({'0':1, '-':0, '+':2})
Sentences.to_csv("./train.csv", index=False)

In [94]:
tree = etree.parse('./news_sentiment_romip2012/test/news_eval_test.xml')  
root = tree.getroot() 
corpus = root
data = []

for sentense in corpus:
    new_sentense = []
    new_sentense.append(sentense.items()[0][1])
#     print sentense.getchildren()
    for field in sentense[:-1]:
        if field.text is None:
            new_sentense.append(None)
        else:
            
            new_sentense.append(field.text.encode('utf-8').strip())
    data.append(new_sentense)

Sentences = pd.DataFrame(np.asarray(data), columns = ['id', 'text', 'label'])
Sentences[['id']] = Sentences[['id']].apply(pd.to_numeric) 
Sentences = Sentences[(Sentences.label == "0") | (Sentences.label == "+") | (Sentences.label == "-")]
Sentences['label'] = Sentences['label'].map({'0':1, '-':0, '+':2})
Sentences.to_csv("./test.csv", index=False)

# Нормализуем данные

In [99]:
RusLem = pymorphy2.MorphAnalyzer()

In [100]:
Sentences = pd.read_csv("./train.csv")
np_sentences = Sentences.values

for sent_idx, sent in tqdm(enumerate(np_sentences)):
    tokens = re.findall('[\w]+',sent[1].decode("utf-8").strip().lower(), re.U)
    tokens = [RusLem.parse(token)[0].normal_form for token in tokens]
    np_sentences[sent_idx][1] = " ".join(tokens)
    
with open("./train_norm", 'w') as res_file:
    pickle.dump(np_sentences, res_file)

3893it [01:03, 65.37it/s]


In [101]:
Sentences = pd.read_csv("./test.csv")
np_sentences = Sentences.values

for sent_idx, sent in tqdm(enumerate(np_sentences)):
    tokens = re.findall('[\w]+',sent[1].decode("utf-8").strip().lower(), re.U)
    tokens = [RusLem.parse(token)[0].normal_form for token in tokens]
    np_sentences[sent_idx][1] = " ".join(tokens)
    
with open("./test_norm", 'w') as res_file:
    pickle.dump(np_sentences, res_file)

4573it [01:17, 42.84it/s]


# Создаем 3 векторных представления -  бинарные вектора, частоты и tf.idf

In [2]:
with open("./train_norm", 'r') as res_file:
    np_sentences_train = pickle.load(res_file)
with open("./test_norm", 'r') as res_file:
    np_sentences_test = pickle.load(res_file)
    
print len(np_sentences_train), len(np_sentences_test)

3893 4573


In [3]:
train_text = [item[1] for item in np_sentences_train]
train_label = [int(item[2]) for item in np_sentences_train]
test_text = [item[1] for item in np_sentences_test]
test_label = [int(item[2]) for item in np_sentences_test]
print len(train_text), len(test_text),  len(train_label), len(test_label)

3893 4573 3893 4573


In [4]:
binary_vectorizer = CountVectorizer(binary=True)
train_binary = binary_vectorizer.fit_transform(train_text).toarray()
test_binary = binary_vectorizer.transform(test_text).toarray()

In [5]:
count_vectorizer = CountVectorizer(binary=False)
train_freq = count_vectorizer.fit_transform(train_text).toarray()
test_freq = count_vectorizer.transform(test_text).toarray()

In [6]:
tfidf_vectorizer = TfidfVectorizer()
train_tfidf = tfidf_vectorizer.fit_transform(train_text).toarray()
test_tfidf = tfidf_vectorizer.transform(test_text).toarray()

# SVM

In [14]:
svm_binary = LinearSVC(max_iter=100000, verbose=2).fit(train_binary, train_label)
svm_freq = LinearSVC(max_iter=100000, verbose=1).fit(train_freq, train_label)
svm_tfidf = LinearSVC(max_iter=100000, verbose=1).fit(train_tfidf, train_label)

[LibLinear][LibLinear][LibLinear]

In [15]:
predict_binary = svm_binary.predict(test_binary)
predict_freq = svm_freq.predict(test_freq)
predict_tfidf = svm_tfidf.predict(test_tfidf)

In [16]:
print predict_binary[:10]
print predict_freq[:10]
print predict_tfidf[:10]

[2 0 0 1 2 0 2 0 0 0]
[2 1 1 1 2 0 2 0 0 0]
[2 0 1 0 0 0 2 0 0 0]


In [17]:
print "Weighted F-measure"
print "Бинарные вектора: ", f1_score(test_label, predict_binary, average='weighted')
print "Частотные вектора: ", f1_score(test_label, predict_freq, average='weighted')
print "Tf-Idf вектора: ", f1_score(test_label, predict_tfidf, average='weighted')
print
print "Micro F-measure"
print "Бинарные вектора: ", f1_score(test_label, predict_binary, average='micro')
print "Частотные вектора: ", f1_score(test_label, predict_freq, average='micro')
print "Tf-Idf вектора: ", f1_score(test_label, predict_tfidf, average='micro')
print 
print "Macro F-measure"
print "Бинарные вектора: ", f1_score(test_label, predict_binary, average='macro')
print "Частотные вектора: ", f1_score(test_label, predict_freq, average='macro')
print "Tf-Idf вектора: ", f1_score(test_label, predict_tfidf, average='macro')

Weighted F-measure
Бинарные вектора:  0.575844001209
Частотные вектора:  0.576135308447
Tf-Idf вектора:  0.597736897068

Micro F-measure
Бинарные вектора:  0.579269626066
Частотные вектора:  0.579706975727
Tf-Idf вектора:  0.612726875137

Macro F-measure
Бинарные вектора:  0.559447225885
Частотные вектора:  0.559523708572
Tf-Idf вектора:  0.575986211597


#  Naive Bayes

In [24]:
mb_binary = BernoulliNB(binarize=None).fit(train_binary, train_label)
mb_freq = MultinomialNB().fit(train_freq, train_label)

In [25]:
predict_binary = mb_binary.predict(test_binary)
predict_freq = mb_freq.predict(test_freq)

In [26]:
print predict_binary[:30]
print predict_freq[:30]

[2 0 0 0 0 0 2 0 0 2 2 2 0 2 2 0 0 0 0 0 0 1 2 0 0 2 0 0 1 2]
[2 0 0 0 0 0 2 0 0 2 2 2 2 2 2 0 0 2 0 0 2 1 2 0 1 2 0 0 1 2]


In [28]:
print "Weighted F-measure"
print "Бинарные вектора (Bernoulli Naive Bayes): ", f1_score(test_label, predict_binary, average='weighted')
print "Частотные вектора (Multinomial Naive Bayes): ", f1_score(test_label, predict_freq, average='weighted')
print
print "Micro F-measure"
print "Бинарные вектора (Bernoulli Naive Bayes): ", f1_score(test_label, predict_binary, average='micro')
print "Частотные вектора (Multinomial Naive Bayes): ", f1_score(test_label, predict_freq, average='micro')
print 
print "Macro F-measure"
print "Бинарные вектора (Bernoulli Naive Bayes): ", f1_score(test_label, predict_binary, average='macro')
print "Частотные вектора (Multinomial Naive Bayes): ", f1_score(test_label, predict_freq, average='macro')

Weighted F-measure
Бинарные вектора (Bernoulli Naive Bayes):  0.556684297921
Частотные вектора (Multinomial Naive Bayes):  0.608023069625

Micro F-measure
Бинарные вектора (Bernoulli Naive Bayes):  0.599387710475
Частотные вектора (Multinomial Naive Bayes):  0.631751585393

Macro F-measure
Бинарные вектора (Bernoulli Naive Bayes):  0.528240378265
Частотные вектора (Multinomial Naive Bayes):  0.584514362325


# Multi-layer Perceptron classifier

In [35]:
mlp_binary = MLPClassifier(hidden_layer_sizes=(1000, 200), 
                           early_stopping=True, max_iter=1000000, verbose=1).fit(train_binary, train_label)

Iteration 1, loss = 0.96027335
Validation score: 0.638462
Iteration 2, loss = 0.38334274
Validation score: 0.717949
Iteration 3, loss = 0.06398220
Validation score: 0.707692
Iteration 4, loss = 0.01790800
Validation score: 0.710256
Iteration 5, loss = 0.01138526
Validation score: 0.707692
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


In [36]:
mlp_tfidf = MLPClassifier(hidden_layer_sizes=(1000, 200), 
                        early_stopping=True, max_iter=1000000, verbose=1).fit(train_tfidf, train_label)

Iteration 1, loss = 1.00165238
Validation score: 0.589744
Iteration 2, loss = 0.54291951
Validation score: 0.674359
Iteration 3, loss = 0.12094150
Validation score: 0.671795
Iteration 4, loss = 0.01908670
Validation score: 0.661538
Iteration 5, loss = 0.01116698
Validation score: 0.653846
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


In [37]:
predict_binary = mlp_binary.predict(test_binary)
predict_tfidf = mlp_tfidf.predict(test_tfidf)

In [38]:
print predict_binary[:30]
print predict_tfidf[:30]

[2 0 0 0 0 0 2 0 0 0 2 2 2 2 2 0 2 0 0 0 0 1 2 0 1 2 0 0 1 2]
[2 0 1 0 0 0 2 0 0 2 2 2 2 2 2 0 2 0 0 0 2 1 2 0 1 2 0 0 1 2]


In [40]:
print "Weighted F-measure"
print "Бинарные вектора: ", f1_score(test_label, predict_binary, average='weighted')
print "Tf-Idf вектора: ", f1_score(test_label, predict_tfidf, average='weighted')
print
print "Micro F-measure"
print "Бинарные вектора: ", f1_score(test_label, predict_binary, average='micro')
print "Tf-idf вектора: ", f1_score(test_label, predict_tfidf, average='micro')
print 
print "Macro F-measure"
print "Бинарные вектора: ", f1_score(test_label, predict_binary, average='macro')
print "Tf-idf вектора: ", f1_score(test_label, predict_tfidf, average='macro')

Weighted F-measure
Бинарные вектора:  0.61959939841
Tf-Idf вектора:  0.609983861535

Micro F-measure
Бинарные вектора:  0.63065821124
Tf-idf вектора:  0.631970260223

Macro F-measure
Бинарные вектора:  0.600634310491
Tf-idf вектора:  0.586892818653
